In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from secret_key import openapi_key, google_api_key, db_user, db_password, db_host, db_name
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.prompts import SemanticSimilarityExampleSelector, FewShotPromptTemplate

from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.prompts.prompt import PromptTemplate

from decimal import Decimal
import datetime

import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# LLM Definition and Invoking

In [8]:
llm = ChatOpenAI(
    model='gpt-4o',
    temperature=0.2,
    api_key = openapi_key
)

In [3]:
# Sample run to check if LLM working
messages = [
    SystemMessage("Act like a psychologist."),
    HumanMessage("What are your thoughts on Nihilism?")
]

result = llm.invoke(messages)

print(result.content)

Nihilism is a philosophical perspective that suggests life lacks inherent meaning, purpose, or intrinsic value. It often leads to the belief that traditional values and beliefs are unfounded, and that existence itself is senseless or absurd. Here are a few thoughts on nihilism from a psychological perspective:

1. **Existential Crisis**: Nihilism can lead to existential crises, where individuals struggle with feelings of meaninglessness and despair. This can be a challenging emotional state, but it can also be an opportunity for personal growth and self-discovery.

2. **Coping Mechanisms**: People who resonate with nihilistic views might develop various coping mechanisms. Some might turn to hedonism, seeking pleasure as a way to fill the perceived void, while others might engage in creative or intellectual pursuits to construct their own meaning.

3. **Resilience and Meaning-Making**: Despite its potentially bleak outlook, confronting nihilistic ideas can lead individuals to develop re

# SQL DataBase

In [9]:
db = SQLDatabase.from_uri(f"mysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE avocadodata (
	`FIELD1` INTEGER NOT NULL, 
	`Date` DATE NOT NULL, 
	`AveragePrice` DECIMAL(4, 2) NOT NULL, 
	`Total_Volume` DECIMAL(11, 2) NOT NULL, 
	`4046_type` DECIMAL(11, 2) NOT NULL, 
	`4225_type` DECIMAL(11, 2) NOT NULL, 
	`4770_type` DECIMAL(10, 2) NOT NULL, 
	`Total_Bags` DECIMAL(11, 2) NOT NULL, 
	`Small_Bags` DECIMAL(11, 2) NOT NULL, 
	`Large_Bags` DECIMAL(10, 2) NOT NULL, 
	`XLarge_Bags` DECIMAL(9, 2) NOT NULL, 
	type VARCHAR(12) NOT NULL, 
	year INTEGER NOT NULL, 
	region VARCHAR(19) NOT NULL
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from avocadodata table:
FIELD1	Date	AveragePrice	Total_Volume	4046_type	4225_type	4770_type	Total_Bags	Small_Bags	Large_Bags	XLarge_Bags	type	year	region
0	2015-12-27	1.33	64236.62	1036.74	54454.85	48.16	8696.87	8603.62	93.25	0.00	conventional	2015	Albany
1	2015-12-20	1.35	54876.98	674.28	44638.81	58.33	9505.56	9408.07	97.49	0.00	conventional	2015	Albany
2	2015-12-13	0.93	118220.22	794.70	109149.

## Inferencing queries

In [10]:
# Creating an SQL chain for inference
chain = create_sql_query_chain(llm, db)

In [11]:
def parse_query(s):
    s = s.rsplit('```')[1]
    s = s.replace('sql\n', '')
    # s = s.replace('\n', '')

    try:
        s = s.replace('SQLQuery:', '')
    except:
        return s
    return s

In [7]:
# Query 1
question_1 = "What was the total volume in the year 2015?"
response_1 = chain.invoke({"question": question_1})
print("Query:", parse_query(response_1))
db.run(parse_query(response_1))

# The response is correct

Query: SELECT SUM(`Total_Volume`) AS `Total_Volume_2015`
FROM avocadodata
WHERE `year` = 2015;



"[(Decimal('4385468662.04'),)]"

In [31]:
# Query 2
question_2 = "How many regions are there?"
response_2 = chain.invoke({"question": question_2})
print("Query:", parse_query(response_2))
db.run(parse_query(response_2))

# The response is correct

Query:  SELECT COUNT(DISTINCT `region`) AS region_count FROM avocadodata;



'[(54,)]'

In [30]:
# Query 3
question_3 = "How has average price changed over the years?"
response_3 = chain.invoke({"question": question_3})
print("Query:", parse_query(response_3))
db.run(parse_query(response_3))

# The response is correct

Query:  SELECT `year`, AVG(`AveragePrice`) AS `AveragePrice` FROM avocadodata GROUP BY `year` ORDER BY `year` LIMIT 5;



"[(2015, Decimal('1.375590')), (2016, Decimal('1.338640')), (2017, Decimal('1.515128')), (2018, Decimal('1.347531'))]"

In [29]:
# Query 4
question_4 = "Which month across the years have seen the highest volume?"
response_4 = chain.invoke({"question": question_4})
print("Query:", parse_query(response_4))
db.run(parse_query(response_4))

# The response is correct

Query:  
SELECT MONTH(`Date`) AS `Month`, SUM(`Total_Volume`) AS `TotalVolume`
FROM avocadodata
GROUP BY `Month`
ORDER BY `TotalVolume` DESC
LIMIT 1;



"[(2, Decimal('1760528797.47'))]"

In [27]:
# Query 5
question_5 = "On what dates did the avocado prices dropped significantly and what were the total number of bags sold on those days?"
response_5 = chain.invoke({"question": question_5})
print("Query:", parse_query(response_5))
db.run(parse_query(response_5))

# The response is correct

Query: SELECT 
    `Date`, 
    `AveragePrice`, 
    `Total_Bags`
FROM 
    avocadodata
WHERE 
    `AveragePrice` < (
        SELECT `AveragePrice` - 0.20
        FROM avocadodata AS prev
        WHERE prev.`Date` < avocadodata.`Date`
        ORDER BY prev.`Date` DESC
        LIMIT 1
    )
ORDER BY 
    `Date` DESC
LIMIT 5;



"[(datetime.date(2018, 3, 25), Decimal('1.08'), Decimal('169974.92')), (datetime.date(2018, 3, 25), Decimal('0.96'), Decimal('162102.31')), (datetime.date(2018, 3, 25), Decimal('1.04'), Decimal('308417.11')), (datetime.date(2018, 3, 25), Decimal('0.73'), Decimal('788607.81')), (datetime.date(2018, 3, 25), Decimal('1.04'), Decimal('1946728.78'))]"

In [34]:
# Query 6
question_6 = "Can you find for each region their preference for small, large or XLarge bags in terms of percentage leading up to 100%?"
response_6 = chain.invoke({"question": question_6})
print("Query:", parse_query(response_6))
db.run(parse_query(response_6))

# The response is correct

Query:  
SELECT 
    `region`, 
    ROUND((SUM(`Small_Bags`) / SUM(`Total_Bags`)) * 100, 2) AS `Small_Bags_Percentage`, 
    ROUND((SUM(`Large_Bags`) / SUM(`Total_Bags`)) * 100, 2) AS `Large_Bags_Percentage`, 
    ROUND((SUM(`XLarge_Bags`) / SUM(`Total_Bags`)) * 100, 2) AS `XLarge_Bags_Percentage`
FROM 
    avocadodata
GROUP BY 
    `region`
LIMIT 5;



"[('Albany', Decimal('83.83'), Decimal('14.55'), Decimal('1.62')), ('Atlanta', Decimal('61.07'), Decimal('37.95'), Decimal('0.97')), ('BaltimoreWashington', Decimal('96.63'), Decimal('2.78'), Decimal('0.59')), ('Boise', Decimal('86.64'), Decimal('13.17'), Decimal('0.19')), ('Boston', Decimal('92.63'), Decimal('6.98'), Decimal('0.39'))]"

In [36]:
# Query 7
question_7 = "Can you find for each region their preference for conventional or organic in terms of percentage leading up to 100%?\
              Each row should represent a single region."
response_7 = chain.invoke({"question": question_7})
print("Query:", parse_query(response_7))
db.run(parse_query(response_7))

# The response is correct

Query:  
SELECT 
    `region`, 
    SUM(CASE WHEN `type` = 'conventional' THEN 1 ELSE 0 END) / COUNT(*) * 100 AS `conventional_percentage`,
    SUM(CASE WHEN `type` = 'organic' THEN 1 ELSE 0 END) / COUNT(*) * 100 AS `organic_percentage`
FROM 
    avocadodata
GROUP BY 
    `region`
LIMIT 5;



"[('Albany', Decimal('50.0000'), Decimal('50.0000')), ('Atlanta', Decimal('50.0000'), Decimal('50.0000')), ('BaltimoreWashington', Decimal('50.0000'), Decimal('50.0000')), ('Boise', Decimal('50.0000'), Decimal('50.0000')), ('Boston', Decimal('50.0000'), Decimal('50.0000'))]"

# Few Shot Learning

In [12]:
few_shots = [
    {'Question': 'Which 5 regions have the most volatile avocado prices year-over-year?',
 'SQLQuery': """
    SELECT region,
           year,
           ROUND(AVG(AveragePrice), 2) AS avg_price,
           ROUND(STDDEV(AveragePrice), 2) AS price_stddev
    FROM avocadodata
    GROUP BY region, year
    ORDER BY price_stddev DESC
    LIMIT 5;
 """,
 'SQLResult': """[('Spokane', 2017, Decimal('1.60'), 0.61), ('Portland', 2017, Decimal('1.43'), 0.61), ('Seattle', 2017, Decimal('1.60'), 0.61), ('PhoenixTucson', 2016, Decimal('1.26'), 0.6), ('SanFrancisco', 2016, Decimal('1.88'), 0.57)]""",
 'Answer': """['Spokane', 'Portland', 'Seattle', 'PhoenixTucson', 'SanFrancisco']"""},


    {'Question': 'Which regions show increasing demand and decreasing average price year over year (possible oversupply)?',
 'SQLQuery': """
    WITH yearly_trends AS (
      SELECT region, year,
             SUM(Total_Volume) AS yearly_volume,
             AVG(AveragePrice) AS yearly_avg_price
      FROM avocadodata
      GROUP BY region, year
    ),
    price_volume_change AS (
      SELECT t1.region,
             t1.year AS year_start,
             t2.year AS year_end,
             t2.yearly_volume - t1.yearly_volume AS volume_change,
             t2.yearly_avg_price - t1.yearly_avg_price AS price_change
      FROM yearly_trends t1
      JOIN yearly_trends t2 ON t1.region = t2.region AND t1.year = t2.year - 1
    )
    SELECT *
    FROM price_volume_change
    WHERE volume_change > 0 AND price_change < 0
    ORDER BY ABS(volume_change * price_change) DESC
    LIMIT 5;
 """,
 'SQLResult': """[('Southeast', 2015, 2016, Decimal('38026268.59'), Decimal('-0.127789')), ('West', 2015, 2016, Decimal('39907015.38'), Decimal('-0.056923')), ('SouthCentral', 2015, 2016, Decimal('13644105.60'), Decimal('-0.085096')), ('Portland', 2015, 2016, Decimal('9368056.53'), Decimal('-0.115192')), ('LosAngeles', 2015, 2016, Decimal('14921609.84'), Decimal('-0.068750'))]""",
 'Answer': """['Southeast', 'West', 'SouthCentral', 'Portland', 'LosAngeles']"""},

    
    {'Question': 'What are the 5 most extreme avocado price outliers compared to the regional average?',
 'SQLQuery': """
    WITH region_stats AS (
      SELECT region,
             AVG(AveragePrice) AS mean_price,
             STDDEV(AveragePrice) AS stddev_price
      FROM avocadodata
      GROUP BY region
    )
    SELECT a.region, a.Date, a.AveragePrice,
           r.mean_price, r.stddev_price
    FROM avocadodata a
    JOIN region_stats r ON a.region = r.region
    WHERE ABS(a.AveragePrice - r.mean_price) > 2 * r.stddev_price
    ORDER BY ABS(a.AveragePrice - r.mean_price) DESC
    LIMIT 5;
 """,
 'SQLResult': """[('Tampa', datetime.date(2017, 4, 16), Decimal('3.17'), Decimal('1.408846'), 0.3405853384705), ('WestTexNewMexico', datetime.date(2016, 10, 16), Decimal('2.93'), Decimal('1.261701'), 0.4963644849685269), ('LasVegas', datetime.date(2016, 10, 2), Decimal('3.03'), Decimal('1.380917'), 0.47326642758671433), ('MiamiFtLauderdale', datetime.date(2017, 3, 12), Decimal('3.05'), Decimal('1.428491'), 0.33599596042836527), ('WestTexNewMexico', datetime.date(2016, 9, 25), Decimal('2.83'), Decimal('1.261701'), 0.4963644849685269)]""",
 'Answer': """['Tampa 2017-04-16', 'WestTexNewMexico 2016-10-16', 'LasVegas 2016-10-02', 'MiamiFtLauderdale 2017-03-12', 'WestTexNewMexico 2016-09-25']"""},




{'Question': 'Which 5 calendar months have the highest average avocado prices across all years?',
 'SQLQuery': """
    SELECT EXTRACT(MONTH FROM Date) AS month,
           ROUND(AVG(AveragePrice), 2) AS avg_price
    FROM avocadodata
    GROUP BY EXTRACT(MONTH FROM Date)
    ORDER BY avg_price DESC
    LIMIT 5;
 """,
 'SQLResult': """[(10, Decimal('1.58')), (9, Decimal('1.57')), (8, Decimal('1.51')), (11, Decimal('1.46')), (7, Decimal('1.46'))]""",
 'Answer': """['10', '9', '8', '11', '7']"""}


]

In [13]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",
                              api_key = openapi_key)

to_vectorize = [" ".join(example.values()) for example in few_shots]

vectorstore = InMemoryVectorStore.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [14]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# example_selector.select_examples({"Question": "Which 5 regions have the least volatile avocado prices year-over-year?"})

In [14]:
example_selector.select_examples({"Question": "Least volatile avocado prices year-over-year?"})

[{'Question': 'Which 5 regions have the most volatile avocado prices year-over-year?',
  'SQLQuery': '\n    SELECT region,\n           year,\n           ROUND(AVG(AveragePrice), 2) AS avg_price,\n           ROUND(STDDEV(AveragePrice), 2) AS price_stddev\n    FROM avocadodata\n    GROUP BY region, year\n    ORDER BY price_stddev DESC\n    LIMIT 5;\n ',
  'SQLResult': "[('Spokane', 2017, Decimal('1.60'), 0.61), ('Portland', 2017, Decimal('1.43'), 0.61), ('Seattle', 2017, Decimal('1.60'), 0.61), ('PhoenixTucson', 2016, Decimal('1.26'), 0.6), ('SanFrancisco', 2016, Decimal('1.88'), 0.57)]",
  'Answer': "['Spokane', 'Portland', 'Seattle', 'PhoenixTucson', 'SanFrancisco']"},
 {'Question': 'Which regions show increasing demand and decreasing average price year over year (possible oversupply)?',
  'SQLQuery': '\n    WITH yearly_trends AS (\n      SELECT region, year,\n             SUM(Total_Volume) AS yearly_volume,\n             AVG(AveragePrice) AS yearly_avg_price\n      FROM avocadodata\n 

# Setting up PromptTemplete using input variables

In [15]:
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [16]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [17]:
chain = create_sql_query_chain(llm, db, prompt=few_shot_prompt)

# Processing data into tables (Beautification!)

In [53]:
aug_q1 = "Generate descriptive statistics for all the numerical columns?"
aug_q1_response = chain.invoke({"question": aug_q1})
print("Query:", parse_query(aug_q1_response))
result = db.run(parse_query(aug_q1_response), include_columns=True)

Query: SELECT 
    ROUND(AVG(`AveragePrice`), 2) AS avg_average_price,
    ROUND(STDDEV(`AveragePrice`), 2) AS stddev_average_price,
    ROUND(MIN(`AveragePrice`), 2) AS min_average_price,
    ROUND(MAX(`AveragePrice`), 2) AS max_average_price,
    ROUND(AVG(`Total_Volume`), 2) AS avg_total_volume,
    ROUND(STDDEV(`Total_Volume`), 2) AS stddev_total_volume,
    ROUND(MIN(`Total_Volume`), 2) AS min_total_volume,
    ROUND(MAX(`Total_Volume`), 2) AS max_total_volume,
    ROUND(AVG(`4046_type`), 2) AS avg_4046_type,
    ROUND(STDDEV(`4046_type`), 2) AS stddev_4046_type,
    ROUND(MIN(`4046_type`), 2) AS min_4046_type,
    ROUND(MAX(`4046_type`), 2) AS max_4046_type,
    ROUND(AVG(`4225_type`), 2) AS avg_4225_type,
    ROUND(STDDEV(`4225_type`), 2) AS stddev_4225_type,
    ROUND(MIN(`4225_type`), 2) AS min_4225_type,
    ROUND(MAX(`4225_type`), 2) AS max_4225_type,
    ROUND(AVG(`4770_type`), 2) AS avg_4770_type,
    ROUND(STDDEV(`4770_type`), 2) AS stddev_4770_type,
    ROUND(MIN(`4770_t

In [54]:
pd.DataFrame(eval(result, {'Decimal': Decimal, 'datetime': datetime}))

,avg_average_price,stddev_average_price,min_average_price,max_average_price,avg_total_volume,stddev_total_volume,min_total_volume,max_total_volume,avg_4046_type,stddev_4046_type,...,min_small_bags,max_small_bags,avg_large_bags,stddev_large_bags,min_large_bags,max_large_bags,avg_xlarge_bags,stddev_xlarge_bags,min_xlarge_bags,max_xlarge_bags
0,1.41,0.4,0.44,3.25,850644.01,3453450.73,84.56,62505646.52,293008.42,1264954.42,...,0.00,13384586.80,54338.09,243959.28,0.00,5719096.61,3106.43,17692.41,0.00,551693.65
